## Purpose of this notebook

- Examine similarities and differences between different raters' reliability report grades
- Calculate Cohen's kappa between pairs of raters
- Identify and print reports where raters strongly disagree (grade of 2 vs grade of 0)

## How to use this notebook

- Run each of the cells in order. Make sure you run Cell 01 first.
- Cell 02 can be used to see how many reports have been graded since a date (YYYY-MM-DD format)
- Cells 03-04 get the proc_ord_id values unique to each report and the names of the persons who have graded reliability reports.
- Cell 05 can be used to examine the reports a pair of graders disagree on
- Cells 06 will release examined reports back to a specified grader for regrading
- Cell 07 can be used to examine and regrade reports marked with a -1 flag

In [1]:
# Cell 01: load libraries
from reliabilityLib import *
from reportMarkingFunctions import *
from google.cloud import bigquery # SQL table interface on Arcus
import pandas
import numpy
import matplotlib.pyplot as plt

client = bigquery.Client()

## Evolution of SLIP over time

In [3]:
# Cell 02:
getGradeCountsSinceDate("2024-08-01")

# Reports 	 Grader Name
99 		 Matt Buczek
171 		 Benjamin Jung
20 		 Sepp Kohler
4 		 Eren Kafadar
374 		 Jenna Schabdach
87 		 Zhiqiang Sha
30 		 Dabriel Zimmerman
208 		 Leila Abdel-Qader
220 		 Harry Hearn

Any graders not in the displayed table have not graded any reports since before 2024-08-01


## Grader agreement on reliability reports

In [ ]:
# Cell 03: Get the list of proc_ord_id values used to identify the reliability reports
procIds = getReliabilityProcOrdIds()

In [ ]:
# Cell 04: Compare the reliability reports for the users we want to evaluate
graders = ['Jenna Schabdach', 
           'Megan M. Himes', 
           'Naomi Shifman', 
           'Alexa DeJean',
           'Julia Katowitz',
           'Matt Buczek',
           'Shivaram Karandikar', 
           'Shreya Gudapati']

# Metric options: "disagreement", "kappa", "kappa2vAll", "kappa0vAll"
df = calculateMetricForGraders(graders, "kappa")
print()
print(df)

In [ ]:
# Cell 05: 
# This is the cell where you can look at the disagreement reports for each pair of users
# User 1: Naomi
# User 2: Jenna
grader1 = "Jenna Schabdach"
grader2 = "Naomi Shifman"
# procIds = getReliabilityProcOrdIds()
# grades1 = getReportsForUser(grader1, procIds)
# grades2 = getReportsForUser(grader2, procIds)
# disagreement = identifyDisagreementReports(grades1, grades2)

# printDisagreementReports(disagreement, grades1, grades2)

In [ ]:
# Cell 06: release a set of your reports back into your queue 
# ONLY USE THIS IF YOU'RE CERTAIN
# releaseReports(grader2, disagreement)

## Release reports of retired graders

In [ ]:
# Edit this list to include the username of anyone who is actively grading
active_graders = ['Jenna Schabdach', 
                  'Harry Hearn', 
                  'Dabriel Zimmerman', 
                  'Sepp Kohler', 
                  'Megan M. Himes', 
                  'Eren Kafadar', 
                  'Matt Buczek', 
                  'Leila Abdel-Qader', 
                  'Laura Mercedes', 
                  'Benjamin Jung', 
                  'Zhiqiang Sha', 
                  'Shivaram Karandikar']

# Run this cell once with the following flag set to True to check your grader list
just_check = True

# Get the list of graders
q = "select distinct grader_name from lab.grader_table_with_metadata;"
graders = client.query(q).to_dataframe()['grader_name'].values

# Drop "Coarse Text Search"
graders = [i for i in graders if "Coarse Text Search" not in i]
print(graders)

# For every grader
for grader in graders:
    # If the grader is no longer active
    if grader not in active_graders:
        # Get the number of reports in their queue
        q = 'select * from lab.grader_table_with_metadata where grader_name = "'+grader+'" and grade = 999 and grade_category = "Unique"'
        grader_df = client.query(q).to_dataframe()
        # Print the number of reports for the user
        print(grader, grader_df.shape)
        # Eventually, delete those entries
        if len(grader_df) > 0 and just_check = False:
            q = 'delete from lab.grader_table_with_metadata where grader_name = "'+grader+'" and grade = 999 and grade_category = "Unique"' 
            job = client.query(q)
            job.result()

## Examine flagged reports

In [ ]:
# Cell 07: regrade skipped reports
# client: A bigquery client object (created in Cell 01)
# skippedGrader: A string of the grader's name (leave blank to review all flagged reports)
skippedGrader = "" # "Naomi Shifman"
regradeSkippedReports(client, skippedGrader)

2/3

Reason report was flagged: developmental venous anomaly with suspicion for gliosis, 2 vs 1?
arcus.procedure_order_narrative
BRAIN MRI, WITHOUT AND WITH CONTRAST: 

CLINICAL INDICATION: 8-year-old, exotropia, left eye. Last MRI with concern for tissue by exit of cranial nerve III on left. Now with progressive symptoms of blurred vision, and worsening drifting of eye. 

TECHNIQUE: Sagittal 3D T1 gradient echo with axial reformations, axial and coronal TSE T2, axial and coronal FLAIR, axial T1, post-contrast sagittal 3D T1 gradient echo with axial reformations, post-contrast axial spin echo T1 with fat suppression, and axial diffusion-weighted imaging were performed on a 1.5 Tesla system. In addition, axial T2 SPACE was performed for evaluation of cranial nerve III. Coronal and sagittal reformations are available for review. 

COMPARISON: MRI brain without contrast, 6/11/2024. 

FINDINGS: Ventricular system normal. Again identified is a developmental venous anomaly within the right f

## For clinician review only

In [3]:
# Cell 08: clinician regrade skipped reports
# client: A bigquery client object (created in Cell 01)
# skippedGrader: A string of the grader's name (leave blank to review all flagged reports)
regradeSkippedReports(client, flag=-2)

1/21

Skipped reason not available.
arcus.procedure_order_narrative
AN ERROR HAS OCCURRED: REPORT 364449975286 CANNOT BE FOUND IN arcus.procedure_order_narrative


UnboundLocalError: local variable 'reportDf' referenced before assignment